## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

Fri Aug 22 19:02:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install Grounding DINO 🦕

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git


/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Total 463 (delta 0), reused 0 (delta 0), pack-reused 463 (from 1)
Receiving objects: 100% (463/463), 12.91 MiB | 14.99 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [ ]:
%cd /content/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn
!sed -i 's/value.type()/value.scalar_type()/g' ms_deform_attn_cuda.cu
!sed -i 's/value.scalar_type().is_cuda()/value.is_cuda()/g' ms_deform_attn_cuda.cu

/content/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn


In [ ]:
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.6 MB/s eta 0:00:00


In [ ]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


## Download Grounding DINO Weights 🏋️

In [ ]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [ ]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


## Download Example Data

In [ ]:
%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data


/content
/content/data


## Load Grounding DINO Model

In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Grounding DINO Demo

In [ ]:
!pip install -q segment_anything

In [ ]:
%cd {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content/weights


In [ ]:
# Descargar el checkpoint de SAM (vit_h)
!mkdir -p /content/weights
!wget -O /content/weights/sam_vit_h_4b8939.pth \
  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


--2025-08-22 19:05:29--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.222.244.122, 52.222.244.102, 52.222.244.33, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.222.244.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘/content/weights/sam_vit_h_4b8939.pth’

/content/weights/sa 100%[===================>]   2.39G   164MB/s    in 21s     

2025-08-22 19:05:50 (117 MB/s) - ‘/content/weights/sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, json
from pathlib import Path
import torch
import numpy as np
import cv2
from PIL import Image

# GroundingDINO
from groundingdino.util.inference import load_model, load_image, predict, annotate

# SAM
from segment_anything import sam_model_registry, SamPredictor

# =========================
# Configuración general
# =========================
CARPETA_IMAGENES        = "/content/data"
CARPETA_SALIDA_IMAGENES = "/content/listos"
CARPETA_SALIDA_COORDS   = "/content/coords"
CARPETA_SALIDA_MASKS    = "/content/masks"

TEXT_PROMPT = [
    "satellite view circle",
    "satellite crop circle",
    "satellite view crop circle",
    "circle"
]

BOX_THRESHOLD  = 0.30 #confianza mínima de la detección.
TEXT_THRESHOLD = 0.30 #similitud mínima entre texto y objeto.
NMS_IOU        = 0.40 #cuánto pueden solaparse dos cajas antes de descartar una

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# modelos
CFG_PATH       = "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
WEIGHTS_PATH   = "/content/weights/groundingdino_swint_ogc.pth"
SAM_MODEL_TYPE = "vit_h"
SAM_CKPT       = "/content/weights/sam_vit_h_4b8939.pth"

# formatos aceptados
EXTS = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp", ".webp"}

# =========================
# Utils
# =========================
def ensure_dirs():
    """Crea las carpetas de salida si no existen."""
    for c in [CARPETA_SALIDA_IMAGENES, CARPETA_SALIDA_COORDS, CARPETA_SALIDA_MASKS]:
        Path(c).mkdir(parents=True, exist_ok=True)

def cxcywh_norm_to_xyxy_abs(boxes_norm: torch.Tensor, W: int, H: int):
    """Convierte coordenadas normalizadas (cx, cy, w, h) a coordenadas absolutas (x1, y1, x2, y2)."""
    scale = torch.tensor([W, H, W, H], dtype=boxes_norm.dtype, device=boxes_norm.device)
    cxcywh_abs = boxes_norm * scale
    cx, cy, w, h = cxcywh_abs.unbind(-1)
    x1 = cx - w/2; y1 = cy - h/2
    x2 = cx + w/2; y2 = cy + h/2
    return torch.stack([x1, y1, x2, y2], dim=-1), cxcywh_abs

def clip_xyxy(xyxy: torch.Tensor, W: int, H: int):
    """Recorta las coordenadas (x1, y1, x2, y2) a los límites de la imagen."""
    xyxy[..., 0::2] = xyxy[..., 0::2].clamp(0, W-1)
    xyxy[..., 1::2] = xyxy[..., 1::2].clamp(0, H-1)
    return xyxy

def nms_torch(xyxy: torch.Tensor, scores: torch.Tensor, iou_thr: float):
    """Aplica Non-Maximum Suppression (NMS) a las detecciones."""
    idxs = scores.argsort(descending=True)
    keep = []
    while idxs.numel() > 0:
        i = idxs[0]; keep.append(i.item())
        if idxs.numel() == 1: break
        rest = idxs[1:]
        xx1 = torch.maximum(xyxy[i,0], xyxy[rest,0])
        yy1 = torch.maximum(xyxy[i,1], xyxy[rest,1])
        xx2 = torch.minimum(xyxy[i,2], xyxy[rest,2])
        yy2 = torch.minimum(xyxy[i,3], xyxy[rest,3])
        w = (xx2-xx1).clamp(min=0); h=(yy2-yy1).clamp(min=0)
        inter = w*h
        area_i = (xyxy[i,2]-xyxy[i,0])*(xyxy[i,3]-xyxy[i,1])
        area_r = (xyxy[rest,2]-xyxy[rest,0])*(xyxy[rest,3]-xyxy[rest,1])
        iou = inter / (area_i + area_r - inter + 1e-8)
        idxs = rest[iou <= iou_thr]
    return torch.tensor(keep, dtype=torch.long)

# =========================
# SAM segmentación
# =========================
def segmentar_con_sam(img_bgr: np.ndarray, boxes_xyxy: np.ndarray):
    """
    Segmenta la imagen utilizando el modelo SAM y las cajas delimitadoras proporcionadas.
    """

    # Convertir a RGB (BGR a RGB)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    H, W = img_rgb.shape[:2]

    # Inicializar SAM
    sam = sam_model_registry[SAM_MODEL_TYPE](checkpoint=SAM_CKPT).to(DEVICE)
    predictor = SamPredictor(sam)
    predictor.set_image(img_rgb)


    if boxes_xyxy.size == 0:
        return [], np.zeros((H, W), np.uint8)

    # Aplicar transformaciones a las cajas
    boxes_t = predictor.transform.apply_boxes_torch(
        torch.from_numpy(boxes_xyxy).to(DEVICE),
        (H, W)
    )
    # Aplicar transformaciones a las máscaras
    with torch.no_grad():
        masks_t, _, _ = predictor.predict_torch(
            point_coords=None, point_labels=None,
            boxes=boxes_t, multimask_output=False
        )


    # Convertir a numpy
    masks_np = masks_t.squeeze(1).detach().cpu().numpy()  # [N,H,W]
    masks_inst = [(m.astype(np.uint8) * 255) for m in masks_np]
    mask_bin = (np.any(masks_np, axis=0).astype(np.uint8) * 255)
    return masks_inst, mask_bin
# =========================
# Pipeline principal
# =========================
def main():
    ensure_dirs()

    print(f"✅ Cargando GroundingDINO\n - CFG: {CFG_PATH}\n - Weights: {WEIGHTS_PATH}")
    model = load_model(CFG_PATH, WEIGHTS_PATH).to(DEVICE).eval()

    rutas = sorted([p for p in Path(CARPETA_IMAGENES).iterdir() if p.suffix.lower() in EXTS])
    if not rutas:
        raise FileNotFoundError(f"No hay imágenes en {CARPETA_IMAGENES}")

    with torch.no_grad():
        for path_img in rutas:
            print(f"\n Procesando: {path_img.name}")
            image_source, image = load_image(str(path_img))  # BGR
            H, W = image_source.shape[:2]

            #detección
            all_boxes, all_scores, all_phrases = [], [], []
            for prompt in TEXT_PROMPT:
                boxes_norm, logits, phrases = predict(
                    model=model, image=image, caption=prompt,
                    box_threshold=BOX_THRESHOLD, text_threshold=TEXT_THRESHOLD
                )
                if boxes_norm is None or len(boxes_norm) == 0:
                    continue
                all_boxes.append(torch.tensor(boxes_norm, dtype=torch.float32))
                all_scores.append(torch.tensor(logits, dtype=torch.float32))
                all_phrases.extend(phrases)

            if all_boxes:
                boxes_norm_cat = torch.cat(all_boxes, dim=0)
                scores_cat = torch.cat(all_scores, dim=0)
                phrases_cat = all_phrases
            else:
                boxes_norm_cat = torch.zeros((0,4), dtype=torch.float32)
                scores_cat = torch.zeros((0,), dtype=torch.float32)
                phrases_cat = []

            xyxy_abs, cxcywh_abs = cxcywh_norm_to_xyxy_abs(boxes_norm_cat, W, H)
            xyxy_abs = clip_xyxy(xyxy_abs, W, H)

            if xyxy_abs.shape[0] > 0:
                keep = nms_torch(xyxy_abs, scores_cat, NMS_IOU)
                xyxy_abs, cxcywh_abs = xyxy_abs[keep], cxcywh_abs[keep]
                boxes_norm_cat, scores_cat = boxes_norm_cat[keep], scores_cat[keep]
                phrases_cat = [phrases_cat[i] for i in keep.tolist()]

            # === guardar imagen anotada ===
            annotated = annotate(image_source, boxes=boxes_norm_cat, logits=scores_cat, phrases=phrases_cat)
            Image.fromarray(annotated).save(str(Path(CARPETA_SALIDA_IMAGENES) / f"detect_{path_img.name}"))

            # === guardar coords en JSON ===
            coords = []
            for i in range(xyxy_abs.shape[0]):
                coords.append({
                    "box_xyxy_abs": [int(v) for v in xyxy_abs[i].tolist()],
                    "confidence": float(scores_cat[i].item()),
                    "label": phrases_cat[i] if i < len(phrases_cat) else ""
                })
            with open(Path(CARPETA_SALIDA_COORDS)/f"detect_{path_img.stem}.json","w") as f:
                json.dump(coords, f, indent=2)

            # === segmentación con SAM ===
            boxes_xyxy_np = xyxy_abs.cpu().numpy().astype(np.float32) if xyxy_abs.numel()>0 else np.zeros((0,4), np.float32)
            masks_inst, mask_bin = segmentar_con_sam(image_source, boxes_xyxy_np)

            # máscara binaria global (opcional, al nivel raíz de masks)
            cv2.imwrite(str(Path(CARPETA_SALIDA_MASKS) / f"{path_img.stem}_mask_bin.png"), mask_bin)

            # carpeta por parche (por imagen)
            carpeta_img = Path(CARPETA_SALIDA_MASKS) / f"{path_img.stem}_masks"
            carpeta_img.mkdir(parents=True, exist_ok=True)

            # fondo marrón oscuro
            BGR_BROWN = (20, 40, 60)

            # guardado de cada máscara y armado del combinado
            combined_bool = np.zeros((H, W), dtype=bool)

            for k, m in enumerate(masks_inst):
                # m es uint8 0/255, lo paso a bool
                mb = (m > 0)
                combined_bool |= mb

                fondo = np.full((H, W, 3), BGR_BROWN, dtype=np.uint8)
                fondo[mb] = (255, 255, 255)
                cv2.imwrite(str(carpeta_img / f"mask_{k:03d}.png"), fondo)
                np.save(str(carpeta_img / f"mask_{k:03d}.npy"), m)

            # imágenes combinadas dentro de la subcarpeta del parche
            # 1) máscara binaria combinada
            combined_mask_u8 = (combined_bool.astype(np.uint8) * 255)
            cv2.imwrite(str(carpeta_img / "combined_mask.png"), combined_mask_u8)

            # 2) overlay combinado (fondo marrón, unión en blanco)
            combined_overlay = np.full((H, W, 3), BGR_BROWN, dtype=np.uint8)
            combined_overlay[combined_bool] = (255, 255, 255)
            cv2.imwrite(str(carpeta_img / "combined_overlay.png"), combined_overlay)

    print("\n✅ Detección + segmentación completadas (con combinados por parche).")

if __name__ == "__main__":
    main()

✅ Cargando GroundingDINO
 - CFG: /content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py
 - Weights: /content/weights/groundingdino_swint_ogc.pth
final text_encoder_type: bert-base-uncased

 Procesando: patch_0_20230801_20240331.png



 Procesando: patch_10_20230801_20240331.png

 Procesando: patch_1_20230801_20240331.png

 Procesando: patch_2_20230801_20240331.png

 Procesando: patch_3_20230801_20240331.png

 Procesando: patch_4_20230801_20240331.png

 Procesando: patch_5_20230801_20240331.png

 Procesando: patch_6_20230801_20240331.png

 Procesando: patch_7_20230801_20240331.png

 Procesando: patch_8_20230801_20240331.png

 Procesando: patch_9_20230801_20240331.png

✅ Detección + segmentación completadas (con combinados por parche).
